## Tehtävä: Oppimisprojekti 2: Kuvantunnistus omilla kuvilla

Tämän projektin tavoitteena on suunnitella ja toteuttaa kuvantunnistusjärjestelmä käyttäen itse otettuja kuvia. Projekti toteutetaan vaiheittain hyödyntäen eri konvoluutioneuroverkkomenetelmiä yksinkertaisesta edistyneempään.

#### Datankäsittely

In [5]:
import os
from PIL import Image
from tensorflow.keras.preprocessing import image_dataset_from_directory

image_folder = '../week2_photodata'

data = image_dataset_from_directory(
    image_folder,
    labels='inferred',
    label_mode='categorical',
    image_size=(100,100)
)
   
for images, labels in data:
    print(f"Batch of images shape: {images.shape}")

Found 513 files belonging to 4 classes.
Batch of images shape: (32, 100, 100, 3)
Batch of images shape: (32, 100, 100, 3)
Batch of images shape: (32, 100, 100, 3)
Batch of images shape: (32, 100, 100, 3)
Batch of images shape: (32, 100, 100, 3)
Batch of images shape: (32, 100, 100, 3)
Batch of images shape: (32, 100, 100, 3)
Batch of images shape: (32, 100, 100, 3)
Batch of images shape: (32, 100, 100, 3)
Batch of images shape: (32, 100, 100, 3)
Batch of images shape: (32, 100, 100, 3)
Batch of images shape: (32, 100, 100, 3)
Batch of images shape: (32, 100, 100, 3)
Batch of images shape: (32, 100, 100, 3)
Batch of images shape: (32, 100, 100, 3)
Batch of images shape: (32, 100, 100, 3)
Batch of images shape: (1, 100, 100, 3)
